In [243]:
#Created: 15.02.2016
#Last modified: 18.02.2016
#               29.02.2016 add Corr and Dis cols
#               25.05.2016 updated input file and created truncated file for testing
#               24.05.2016 tested reading in file produced in birdhouse analogs process

In [240]:
import numpy as np
import pandas as pd
import collections
import os

In [241]:
num_analogues = 20 #number of analogues searched for
num_cols = 3 #dateAnlg, Dis, Corr

In [242]:
#To read file produced in birdhouse
#dfS = pd.DataFrame()
#analogs = 'http://birdhouse-lsce.extra.cea.fr:8090/wpsoutputs/flyingpigeon/output_txt-0797016c-378e-11e6-91dd-41d8cd554993.txt'
#dfS = pd.read_csv(analogs, delimiter=r"\s+", index_col=0)
#dfS.head()

,date.an1,date.an2,date.an3,date.an4,date.an5,date.an6,date.an7,date.an8,date.an9,date.an10,...,cor11,cor12,cor13,cor14,cor15,cor16,cor17,cor18,cor19,cor20
date,,,,,,,,,,,,,,,,,,,,,
20130715,19560708,19570625,19570626,19570702,19560707,19570701,19570627,19570728,19560709,19570624,...,-0.297310,0.281411,0.414869,0.263460,-0.033923,-0.368161,-0.173413,-0.098069,0.004391,0.028584
20130716,19570626,19570702,19570703,19560709,19570627,19570625,19570704,19570729,19570701,19560708,...,-0.053174,0.340757,-0.257390,-0.169466,-0.079024,-0.049076,-0.222097,0.067864,-0.146098,-0.002266
20130717,19570627,19570704,19570703,19570702,19570626,19560710,19570705,19570730,19570628,19570701,...,-0.309011,0.089543,-0.115584,-0.218824,-0.268240,0.058822,-0.126518,0.143484,-0.248070,0.282629
20130718,19570704,19570705,19570628,19570627,19570703,19570731,19560624,19570702,19560711,19570730,...,-0.099006,-0.167025,0.006505,0.149398,0.106216,0.345971,-0.146334,0.126187,-0.125393,-0.002311
20130719,19560625,19570628,19570801,19570629,19570705,19560624,19570627,19570704,19570706,19560626,...,0.120918,0.276314,0.256931,0.414601,0.373720,0.341264,0.250203,0.170120,0.246184,0.120446


In [244]:
# Create dataframe from analogues file
# Latest file: /home/estimr1/yiou/estimr1/NCEP/slpano-NA.analog30rms.1d.all1948-2016.dat
# 2015: /home/estimr1/yiou/estimr1/NCEP/slpano-NA.analog30rms.1d.all1948-2015.dat
# Note that last day in 2015 file is 20151225 (and NOT 20151231) because that is the last sunday of Dec 2015.

dfS = pd.DataFrame()
dfS = pd.read_csv('/home/estimr1/yiou/estimr1/NCEP/slpano-NA.analog30rms.1d.all1948-2016.dat', delimiter=' ', index_col=0)
#dfS.set_index('Date', inplace=True)

dfS.head(5)

#dfS.index.name

,date.an1,date.an2,date.an3,date.an4,date.an5,date.an6,date.an7,date.an8,date.an9,date.an10,...,cor11,cor12,cor13,cor14,cor15,cor16,cor17,cor18,cor19,cor20
date,,,,,,,,,,,,,,,,,,,,,
19480101,19560126,19850131,20110114,19551224,19850130,19980114,19521221,20001211,19681220,19871224,...,0.496248,0.621168,0.378193,0.594238,0.405156,0.642016,0.371728,0.609399,0.574232,0.360594
19480102,20001211,19681221,19551224,20110112,19560126,19741220,19600121,19681220,19521221,19850131,...,0.555860,0.495064,0.675751,0.473601,0.329321,0.664506,0.592903,0.461086,0.557974,0.495405
19480103,19491206,19741222,19750110,19851222,19641231,19921215,19741221,19931217,19950124,19491207,...,0.591253,0.571272,0.472771,0.424513,0.647315,0.524868,0.521988,0.652662,0.599281,0.510727
19480104,20031217,19891221,19921215,20020121,19510114,19931218,19921216,19741224,19931217,19851222,...,0.452602,0.626009,0.594023,0.556825,0.584074,0.440337,0.448684,0.480334,0.426003,0.356706
19480105,19971210,19971209,19971211,19591230,19931217,19591228,19591229,19950129,20020121,19550120,...,0.605750,0.573455,0.519351,0.550601,0.461024,0.335482,0.569624,0.579975,0.482314,0.523988


In [245]:
dfS.tail(5)

,date.an1,date.an2,date.an3,date.an4,date.an5,date.an6,date.an7,date.an8,date.an9,date.an10,...,cor11,cor12,cor13,cor14,cor15,cor16,cor17,cor18,cor19,cor20
date,,,,,,,,,,,,,,,,,,,,,
20160613,19770514,19580622,19960703,19680620,19880531,19480616,19880701,19540606,19680621,19880601,...,0.511035,0.515002,0.474934,0.573175,0.512184,0.539033,0.566547,0.479236,0.434792,0.414608
20160614,19580622,19880702,19880701,19580623,20120606,19540609,19580621,20090705,20100608,20120607,...,0.561573,0.599886,0.575354,0.578218,0.512706,0.604387,0.643425,0.578632,0.570656,0.510388
20160615,20030702,20100608,19580622,19580623,19540704,19970622,20090607,19880702,19490524,20120607,...,0.596715,0.644501,0.606516,0.574877,0.601888,0.506091,0.613552,0.655912,0.569119,0.611980
20160616,19970622,19580623,20090707,19580624,19530603,20090708,19990705,20030702,19580622,19540705,...,0.487257,0.369651,0.510639,0.496947,0.445235,0.423884,0.529484,0.545829,0.458409,0.338569
20160617,20090708,20090707,20090709,19970623,20050605,19880604,19990706,19990705,19660624,19510626,...,0.516598,0.462850,0.429987,0.586695,0.542526,0.443260,0.354462,0.551617,0.478514,0.535178


In [246]:
dfS.index

Int64Index([19480101, 19480102, 19480103, 19480104, 19480105, 19480106,
            19480107, 19480108, 19480109, 19480110, 
            ...
            20160608, 20160609, 20160610, 20160611, 20160612, 20160613,
            20160614, 20160615, 20160616, 20160617],
           dtype='int64', name=u'date', length=25006)

In [247]:
len(dfS.index)

25006

In [248]:
#Chop off 2016 rows NO!!! there will still be 2016 dates found as analogues, which messes up js script
#dfS = dfS[dfS.index <= 20151231]

In [249]:
#SAVE DATE INDEX
#DF['id2'] = DF.index
dfS['dateRef'] = dfS.index
dfS['dateRef'].head(10)

date
19480101    19480101
19480102    19480102
19480103    19480103
19480104    19480104
19480105    19480105
19480106    19480106
19480107    19480107
19480108    19480108
19480109    19480109
19480110    19480110
Name: dateRef, dtype: int64

In [250]:
#TRANSPOSE ANLG DATE COLS INTO SINGLE COL

#Define temporary df containing only anlg dates cols
df_anlg = dfS.iloc[:, 0:20]

#remove index name before transpose (stack)
df_anlg.index.name = ""

#transpose (stack)into single col
dateStack = df_anlg.stack()
#dateStack

In [251]:
#TRANSPOSE DIS COLS INTO SINGLE COL

#Define temporary df containing only dis cols
df_dis = dfS.iloc[:, 20:40]

#remove index name before transpose (stack)
df_dis.index.name = ""

#transpose (stack) into single col and take absolute value
disStack = df_dis.stack().abs()
disStack.head(5)

19480101  dis1    523.709383
          dis2    586.884633
          dis3    623.145003
          dis4    632.450071
          dis5    635.355183
dtype: float64

In [252]:
#TRANSPOSE CORR COLS INTO SINGLE COL

#Define temporary df containing only corr cols
df_corr = dfS.iloc[:, 40:60]

#remove index name before transpose (stack)
df_corr.index.name = ""

#transpose (stack) into single col
corrStack = df_corr.stack()
corrStack.head(5)

19480101  cor1    0.708439
          cor2    0.676750
          cor3    0.640156
          cor4    0.485716
          cor5    0.519602
dtype: float64

In [253]:
#BUILD NEW DF IN CORRECT FORMAT

#CREATE DF OF CORRECT DIMENSIONS (n x num_cols)
df_all = dfS.iloc[:, 0:num_cols]
#rename cols
df_all.columns = ['dateAnlg', 'Dis', 'Corr']

In [254]:
#dfS.index.tolist()
df_all.head(5)

,dateAnlg,Dis,Corr
,,,
19480101,19560126,19850131,20110114
19480102,20001211,19681221,19551224
19480103,19491206,19741222,19750110
19480104,20031217,19891221,19921215
19480105,19971210,19971209,19971211


In [255]:
df_all = df_all.set_index(dfS['dateRef'] )
df_all.head(10)

,dateAnlg,Dis,Corr
dateRef,,,
19480101,19560126,19850131,20110114
19480102,20001211,19681221,19551224
19480103,19491206,19741222,19750110
19480104,20031217,19891221,19921215
19480105,19971210,19971209,19971211
19480106,20020122,19710120,19931217
19480107,19781214,19591227,19510112
19480108,20040114,19591228,19781214
19480109,20021230,20020121,19931217


In [256]:
#REPLICATE INDEX COL 20 TIMES PER ROW

#http://stackoverflow.com/questions/26777832/replicating-rows-in-a-pandas-data-frame-by-a-column-value
#df2 = df.loc[np.repeat(df.index.values,df.n)]
df_all = df_all.loc[np.repeat(df_all.index.values,num_analogues)]

In [257]:
#REPLACE COLS WITH STACKED ANLG DATES, DIS AND CORR
df_all.iloc[:,0] = list(dateStack)
df_all.iloc[:,1] = list(disStack)
df_all.iloc[:,2] = list(corrStack)
df_all.head(5)

,dateAnlg,Dis,Corr
dateRef,,,
19480101,19560126,523.709383,0.708439
19480101,19850131,586.884633,0.676750
19480101,20110114,623.145003,0.640156
19480101,19551224,632.450071,0.485716
19480101,19850130,635.355183,0.519602


In [258]:
df_all.shape

(500120, 3)

In [259]:
df_all.tail(5)

,dateAnlg,Dis,Corr
dateRef,,,
20160617,19570519,505.247492,0.443260
20160617,19890627,510.247497,0.354462
20160617,19510628,512.333517,0.551617
20160617,19980628,515.598167,0.478514
20160617,19910618,515.688414,0.535178


In [260]:
dfS_select = df_all[df_all.index <= 19600101]

In [261]:
dfS_select.tail(10)

,dateAnlg,Dis,Corr
dateRef,,,
19600101,20040109,797.287846,0.699930
19600101,20040107,801.705454,0.738068
19600101,19750128,811.387152,0.620306
19600101,19541212,813.104730,0.546885
19600101,20001202,825.934345,0.645723
19600101,19740113,825.959133,0.783222
19600101,19521220,837.784433,0.566982
19600101,19661228,838.124823,0.695898
19600101,20001203,840.777348,0.549039


In [76]:
#SAVE TO CSV FILE

#df_all.to_csv('analogues_19480101_20151225.json', float_format='%.2f', sep='\t')

#df_all.to_csv('analogues_19480101_20160520.json', float_format='%.2f', sep='\t')

#dfS_select.to_csv('analogues_select.json', float_format='%.2f', sep='\t')

In [77]:
pwd

u'/homel/cnangini/PROJECTS/analogues'

In [86]:
print df_all['Corr'].min(), df_all['Corr'].max()

-0.0773818303236 0.952985818969


In [87]:
print df_all['Dis'].min(), df_all['Dis'].max()

284.431297359 1369.41118104


In [31]:
#check 1970-1979
df_check =  df_all[ (df_all['dateAnlg'] >= 19700101) & (df_all['dateAnlg'] <= 19791231) ]


#((df['column1']=='banana') & (df['colour']=='green'))

In [32]:
df_check.shape

(75935, 3)

In [33]:
df_check =  df_all[ (df_all['dateAnlg'] >= 20100101) & (df_all['dateAnlg'] <= 20191231) ]

In [34]:
df_check.shape

(44554, 3)

In [35]:
#cut file for testing purposes
df_select = df_all.iloc[19000:19100,:]

In [36]:
df_select.shape

(100, 3)

In [37]:
df_select.tail(10)

,dateAnlg,Dis,Corr
dateRef,,,
19500812,19840828,449.988840,0.546204
19500812,19590909,453.044659,0.612506
19500812,20030819,453.638210,0.559760
19500812,19970801,457.039066,0.597291
19500812,19980815,459.252587,0.534387
19500812,19990829,466.308358,0.580896
19500812,19510820,466.860763,0.727749
19500812,20130816,467.150673,0.623782
19500812,19990717,471.338490,0.571134


In [38]:
df_chunk = df_all.iloc[149000:149100,:]
df_chunk2 = df_all.iloc[106000:106100,:]

In [39]:
df_select = df_select.append(df_chunk)
df_select = df_select.append(df_chunk2)
#df_select = pd.concat([df_select,df_chunk])

In [40]:
df_select.shape

(300, 3)

In [41]:
#SAVE SELECTION TO TSV FILE

#df_select.to_csv('analogues_reformat_all_select.tsv', float_format='%.2f', sep='\t')

In [42]:
print df_all['Corr'].min(), df_all['Corr'].max()

-0.0773818303236 0.952985818969


In [43]:
print df_all['Dis'].min(), df_all['Dis'].max()

284.431297359 1369.41118104


In [44]:
df_select['Corr'].max()
df_select['Corr'].min()
print df_select['Corr'].min(), df_select['Corr'].max()

0.124791552993 0.821019151951


In [45]:
df_select['Dis'].max()
df_select['Dis'].min()
print df_select['Dis'].min(), df_select['Dis'].max()

350.217929418 696.281040107
